<a href="https://colab.research.google.com/github/vektor8891/llm/blob/main/projects/26_prompt_engineering/26_prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install -qq ibm-watsonx-ai
# ! pip install -qq langchain
# ! pip install -qq langchain-ibm

# In-Context Engineering and Prompt Templates

## Setup LLM

In [2]:
from langchain_ibm.llms import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from google.colab import userdata

def llm_model(prompt_txt=None, params=None):
    model_id = 'mistralai/mistral-small-3-1-24b-instruct-2503'

    default_params = {
        "max_new_tokens": 256,
        "min_new_tokens": 0,
        "temperature": 0.5,
        "top_p": 0.2,
        "top_k": 1
    }

    if params:
        default_params.update(params)

    parameters = {
        GenParams.MAX_NEW_TOKENS: default_params["max_new_tokens"],  # this controls the maximum number of tokens in the generated output
        GenParams.MIN_NEW_TOKENS: default_params["min_new_tokens"], # this controls the minimum number of tokens in the generated output
        GenParams.TEMPERATURE: default_params["temperature"], # this randomness or creativity of the model's responses
        GenParams.TOP_P: default_params["top_p"],
        GenParams.TOP_K: default_params["top_k"]
    }

    api_key = userdata.get('IBM_CLOUD_API_KEY')

    credentials = {
      "url": userdata.get("WATSONX_URL"),
      "apikey": userdata.get('IBM_CLOUD_API_KEY')
  }

    project_id = userdata.get("WATSONX_PROJECT_ID")

    mixtral_llm = WatsonxLLM(
        model_id=model_id,
        url=userdata.get("WATSONX_URL"),
        apikey=userdata.get('IBM_CLOUD_API_KEY'),
        project_id=userdata.get("WATSONX_PROJECT_ID"),
        params=parameters
    )

    if prompt_txt:
        response = mixtral_llm.invoke(prompt_txt)
        return response
    else:
        return mixtral_llm

In [3]:
GenParams().get_example_values()

{'decoding_method': 'sample',
 'length_penalty': {'decay_factor': 2.5, 'start_index': 5},
 'temperature': 0.5,
 'top_p': 0.2,
 'top_k': 1,
 'random_seed': 33,
 'repetition_penalty': 2,
 'min_new_tokens': 50,
 'max_new_tokens': 200,
 'stop_sequences': ['fail'],
 'time_limit': 600000,
 'truncate_input_tokens': 200,
 'prompt_variables': {'object': 'brain'},
 'return_options': {'input_text': True,
  'generated_tokens': True,
  'input_tokens': True,
  'token_logprobs': True,
  'token_ranks': False,
  'top_n_tokens': False}}

## Prompt engineering

### First basic prompt

In [4]:
params = {
    "max_new_tokens": 128,
    "min_new_tokens": 10,
    "temperature": 0.5,
    "top_p": 0.2,
    "top_k": 1
}

prompt = "The wind is"

response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: The wind is

response :  blowing at 20 mph. The wind chill is 10 degrees Fahrenheit. What is the temperature?

The wind chill is a measure of how cold it feels outside due to the combination of temperature and wind speed. It does not change the actual air temperature. Therefore, if the wind chill is 10 degrees Fahrenheit, it means that the actual temperature feels like 10 degrees Fahrenheit due to the wind, but the actual air temperature is not provided in the information given.

To determine the actual temperature, we would need additional information or a wind chill chart/table that correlates wind speed and temperature to wind



### Zero-shot prompt

In [5]:
prompt = """Classify the following statement as true or false:
            'The Eiffel Tower is located in Berlin.'

            Answer:
"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Classify the following statement as true or false: 
            'The Eiffel Tower is located in Berlin.'

            Answer:


response :             False

            The Eiffel Tower is actually located in Paris, France.



### One-shot prompt

In [6]:
params = {
    "max_new_tokens": 20,
    "temperature": 0.1,
}

prompt = """Here is an example of translating a sentence from English to French:

            English: “How is the weather today?”
            French: “Comment est le temps aujourd'hui?”

            Now, translate the following sentence from English to French:

            English: “Where is the nearest supermarket?”

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Here is an example of translating a sentence from English to French:

            English: “How is the weather today?”
            French: “Comment est le temps aujourd'hui?”
            
            Now, translate the following sentence from English to French:
            
            English: “Where is the nearest supermarket?”
            


response :             French: “Où est le supermarché le plus proche?”

In this exercise,



### Few-shot prompt

In [7]:
params = {
    "max_new_tokens": 10,
}

prompt = """Here are few examples of classifying emotions in statements:

            Statement: 'I just won my first marathon!'
            Emotion: Joy

            Statement: 'I can't believe I lost my keys again.'
            Emotion: Frustration

            Statement: 'My best friend is moving to another country.'
            Emotion: Sadness

            Now, classify the emotion in the following statement. Return the emotion only:
            Statement: 'That movie was so scary I had to cover my eyes.’


"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Here are few examples of classifying emotions in statements:

            Statement: 'I just won my first marathon!'
            Emotion: Joy
            
            Statement: 'I can't believe I lost my keys again.'
            Emotion: Frustration
            
            Statement: 'My best friend is moving to another country.'
            Emotion: Sadness
            
            Now, classify the emotion in the following statement. Return the emotion only:
            Statement: 'That movie was so scary I had to cover my eyes.’
            



response : Fear



### Chain-of-thought (CoT) prompting

In [8]:
params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
}

prompt = """Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples.
            How many apples are there now?’

            Break down each step of your calculation

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples. 
            How many apples are there now?’

            Break down each step of your calculation



response : 1. **Initial Number of Apples**: The store starts with 22 apples.

2. **Apples Sold**: The store sold 15 apples.
   - Calculation: \( 22 - 15 = 7 \)
   - So, after selling 15 apples, the store has 7 apples left.

3. **New Delivery**: The store received a new delivery of 8 apples.
   - Calculation: \( 7 + 8 = 15 \)
   - So, after the new delivery, the store has 15 apples.

Therefore, the store now has **15 apples**.



### Self-consistency

In [9]:
params = {
    "max_new_tokens": 512,
}

prompt = """When I was 6, my sister was half of my age. Now I am 70, what age is my sister?

            Provide three independent calculations and explanations, then determine the most consistent result.

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: When I was 6, my sister was half of my age. Now I am 70, what age is my sister?

            Provide three independent calculations and explanations, then determine the most consistent result.



response : ### Calculation 1:
1. When you were 6, your sister was half your age, so she was 3.
2. The age difference between you and your sister is 6 - 3 = 3 years.
3. Now that you are 70, your sister's age is 70 - 3 = 67.

### Calculation 2:
1. When you were 6, your sister was half your age, so she was 3.
2. The age difference between you and your sister is constant.
3. Now that you are 70, your sister's age is 70 - 3 = 67.

### Calculation 3:
1. When you were 6, your sister was half your age, so she was 3.
2. The age difference between you and your sister is 6 - 3 = 3 years.
3. Now that you are 70, your sister's age is 70 - 3 = 67.

### Conclusion:
All three calculations consistently show that your sister is 67 years old.



## Applications

In [10]:
from langchain_core.prompts import PromptTemplate

template = """Tell me a {adjective} joke about {content}.
"""
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['adjective', 'content'], input_types={}, partial_variables={}, template='Tell me a {adjective} joke about {content}.\n')

In [11]:
prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.\n'

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_ibm import WatsonxLLM

params = {
    "max_new_tokens": 256,
    "temperature": 0.5,
}

mixtral_llm = llm_model(prompt_txt=None, params=params)

response = prompt | mixtral_llm
response = response.invoke(input = {"adjective": "funny", "content": "chickens"})

print(response)

What do you call a chicken looking at a salad? A Caesar.


### Question answering

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_ibm import WatsonxLLM

content = """
        The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets.
        The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid.
        The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.
"""

question = "Which planets in the solar system are rocky and solid?"

template = """
            Answer the {question} based on the {content}.
            Respond "Unsure about answer" if not sure about the answer.

            Answer:

"""
prompt = PromptTemplate.from_template(template)
response = prompt | mixtral_llm
response = response.invoke(input = {"question":question ,"content": content})
print(response)

The rocky and solid planets in the solar system are Mercury, Venus, Earth, and Mars.


### Text classification

In [22]:
text = """
        The concert last night was an exhilarating experience with outstanding performances by all artists.
"""

categories = "Entertainment, Food and Dining, Technology, Literature, Music."

template = """
            Classify the {text} into one of the {categories}.

            Category:

"""
prompt = PromptTemplate.from_template(template)
response = prompt | mixtral_llm
response = response.invoke(input = {"text":text ,"categories": categories})
print(response)

            Music


### Code generation

In [23]:
description = """
        Retrieve the names and email addresses of all customers from the 'customers' table who have made a purchase in the last 30 days.
        The table 'purchases' contains a column 'purchase_date'
"""

template = """
            Generate an SQL query based on the {description}

            SQL Query:

"""
prompt = PromptTemplate.from_template(template)
response = prompt | mixtral_llm
response = response.invoke(input = {"description":description})
print(response)

            ```sql
            SELECT c.name, c.email
            FROM customers c
            JOIN purchases p ON c.customer_id = p.customer_id
            WHERE p.purchase_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY);
            ```

        This query joins the 'customers' table with the 'purchases' table on the 'customer_id' column and filters the results to include only those customers who have made a purchase in the last 30 days. The `DATE_SUB(CURDATE(), INTERVAL 30 DAY)` function is used to calculate the date 30 days ago from the current date.


### Role playing

In [25]:
from langchain_core.prompts import PromptTemplate
from langchain_ibm import WatsonxLLM

role = """
        game master
"""

tone = "engaging and immersive"

template = """
            You are an expert {role}. I have this question {question}. I would like our conversation to be {tone}.

            Answer:

"""
prompt = PromptTemplate.from_template(template)
response = prompt | mixtral_llm
response = response.invoke(input = {"role": role, "question": "What is my quest?", "tone": tone})
print(response)

            Ah, brave adventurer, you stand before me, eyes gleaming with curiosity and heart pounding with anticipation. The question you pose is one that has echoed through the halls of time, whispered in the winds, and etched into the very fabric of our world. "What is my quest?"

            Let us embark on a journey of discovery, for the path to your quest is as unique as the stars that guide us. Close your eyes and take a deep breath. Feel the cool night air fill your lungs, and listen to the distant hoot of an owl. Now, open your mind and let the visions come.

            Do you see a ancient, crumbling parchment, its edges frayed and ink faded, yet the words still legible? Or perhaps a mysterious, glowing orb that pulses with an otherworldly energy? Maybe you envision a desperate plea from a stranger in need, their voice echoing through the mist.

            Share with me what you see, and together we shall unravel the threads of your destiny. For remember, adventurer, the q

### Game Master Chatbot

In [28]:
while True:
    query = input("Question: ")

    if query.lower() in ["quit","exit","bye"]:
        print("Answer: Goodbye!")
        break

    response = prompt | mixtral_llm
    response = response.invoke(input = {"role": role, "question": query, "tone": tone})
    print("Answer: ", response)

Question: who am i?
Answer:              Welcome, traveler, to this realm of mystery and self-discovery. I am the enigmatic guide, here to help you unravel the threads of your existence. To begin our journey, let us embark on a series of questions and scenarios that will illuminate the path to your true identity.

            First, close your eyes and imagine yourself standing at the edge of a vast, ancient forest. The trees are towering and wise, their leaves whispering secrets in the wind. As you take your first steps into the woods, what do you see, hear, and feel?

            Describe your surroundings and any sensations or thoughts that come to mind. From there, we shall delve deeper into the heart of the mystery that is you.
Question: where ami i?
Answer:              Ah, a mystery! I do love a good puzzle. Let's embark on this journey together. To begin, could you please provide me with a bit more context? Where were you last? What were you doing? Any peculiar sights, sounds, 

KeyboardInterrupt: Interrupted by user

## Exercises

### Exercise 1: Change parameters for the LLM

In [29]:
params = {
    "max_new_tokens": 128,
    "min_new_tokens": 100,
    "temperature": 1,
    "top_p": 0.1,
    "top_k": 1
}

prompt = "The wind is"

response = llm_model(prompt, params)
print(response)

 blowing at 20 mph. The wind chill is 10 degrees Fahrenheit. What is the temperature?

The wind chill is a measure of how cold it feels outside due to the combination of temperature and wind speed. It does not change the actual air temperature. Therefore, if the wind chill is 10 degrees Fahrenheit, it means that the actual temperature feels like 10 degrees Fahrenheit due to the wind, but the actual air temperature is not provided in the information given.

To determine the actual temperature, we would need additional information or a wind chill chart/table that correlates wind speed and temperature to wind


### Exercise 2: Observe how LLM thinks

In [37]:
from langchain.globals import set_debug

set_debug(True)

content = """
        The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets.
        The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid.
        The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.
"""

question = "Which planets in the solar system are rocky and solid?"

template = """
            Answer the {question} based on the {content}.
            Respond "Unsure about answer" if not sure about the answer.

            Answer:

"""
prompt = PromptTemplate.from_template(template)
response = prompt | mixtral_llm
response = response.invoke(input = {"question":question ,"content": content})
print(response)
set_debug(False)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Which planets in the solar system are rocky and solid?",
  "content": "\n        The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets.\n        The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid.\n        The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.\n"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "Which planets in the solar system are rocky and solid?",
  "content": "\n        The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets.\n        The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid.\n        The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.\n"
}
[chain/end] [chain:Runnable

### Exercise 3: Revise the text classification agent to one-shot learning

In [39]:
example_text = """
               Last week's book fair was a delightful gathering of authors and readers, featuring discussions and book signings.
               """

example_category = "Literature"

text = """
       The concert last night was an exhilarating experience with outstanding performances by all artists.
       """

categories = "Entertainment, Food and Dining, Technology, Literature, Music."

template = """
           Example:
           Text: {example_text}
           Category: {example_category}

           Now, classify the following text into one of the specified categories. Return the category only: {categories}

           Text: {text}

           Category:

           """
prompt = PromptTemplate.from_template(template)
llm_chain = prompt | mixtral_llm
response = llm_chain.invoke(input = {"example_text": example_text, "example_category":example_category ,"categories": categories, "text":text})
print(response)

 Music


In [40]:
! pip freeze > requirements.txt